In [1]:
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1)) ## Check the shape
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1)) ## Check the shape
test_images = test_images.astype("float32") / 255
print(len(train_images))
print(len(train_labels))

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
60000
60000


In [3]:
train_images_1 = []
train_labels_1 = []
train_images_2 = []
train_labels_2 = []


digit_counts = {digit: 0 for digit in range(5, 10)}

for i, digit in enumerate(train_labels):
    if digit <= 4:
        train_images_1.append(train_images[i])
        train_labels_1.append(digit)
    elif 5 <= digit <= 9 and digit_counts[digit] < 10:
        train_images_2.append(train_images[i])
        train_labels_2.append(digit)
        digit_counts[digit] += 1


print(len(train_images_1))
print(len(train_labels_1)) #Ensuring same length, and totaling about 30k.

print(len(train_images_2))
print(len(train_labels_2)) #Ensuring 50 count (10 of each digit > 4)

30596
30596
50
50


In [4]:
test_images_1 = []
test_labels_1 = []
test_images_2 = []
test_labels_2 = []

for i, digit in enumerate(test_labels):
    if digit <= 4:
      test_images_1.append(test_images[i])
      test_labels_1.append(digit)
    else:
      test_images_2.append(test_images[i])
      test_labels_2.append(digit)

print(len(test_images_1))
print(len(test_labels_1))

print(len(test_images_2))
print(len(test_labels_2)) #Ensuring same length, total 5k each.

5139
5139
4861
4861


In [5]:
#Shuffling training set 1 and 2:
import numpy as np

perm_1 = np.random.permutation(len(train_images_1))
train_images_1 = np.array(train_images_1)
train_labels_1 = np.array(train_labels_1) #Need to be numpy arrays

perm_2 = np.random.permutation(len(train_images_2))
train_images_2 = np.array(train_images_2)
train_labels_2 = np.array(train_labels_2) #Need to be numpy arrays

train_images_1 = train_images_1[perm_1]
train_labels_1 = train_labels_1[perm_1]

train_images_2 = train_images_2[perm_2]
train_labels_2 = train_labels_2[perm_2]



#Making test_data a numpy array so they can have shape
test_images_1 = np.array(test_images_1)
test_images_2 = np.array(test_images_2)
test_labels_1 = np.array(test_labels_1)
test_labels_2 = np.array(test_labels_2)

print(len(train_images_1))
print(len(train_images_2))

30596
50


In [6]:
#Creating validation data (20% of train)
val_images_1 = train_images_1[:int(.2*len(train_images_1))] #Takes first 20%
train_images_1 = train_images_1[int(.2*len(train_images_1)):] #Assigns training images last 80%

val_labels_1 = train_labels_1[:int(.2*len(train_labels_1))] #Takes first 20%
train_labels_1 = train_labels_1[int(.2*len(train_labels_1)):] #Assigns training labels last 80%

val_images_2 = train_images_2[:int(.2*len(train_images_2))] #Repeat for 2
train_images_2 = train_images_2[int(.2*len(train_images_2)):]

val_labels_2 = train_labels_2[:int(.2*len(train_labels_2))]
train_labels_2 = train_labels_2[int(.2*len(train_labels_2)):]

In [7]:
#Model 1
inputs = keras.Input(shape=(28, 28, 1)) ## Different from densenet input
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model_1 = keras.Model(inputs=inputs, outputs=outputs)

model_1.compile(loss="SparseCategoricalCrossentropy", optimizer="rmsprop",
metrics=["accuracy"])

#Creating ModelCheckpoint callback
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="Model_1.keras",
save_best_only=True,
monitor="val_loss")
]

history = model_1.fit(train_images_1, train_labels_1,
epochs=30,
validation_data=(val_images_1,val_labels_1),
callbacks=callbacks)

Epoch 1/30
765/765 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9258 - loss: 0.2087 - val_accuracy: 0.9912 - val_loss: 0.0278
Epoch 2/30
765/765 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9926 - loss: 0.0222 - val_accuracy: 0.9948 - val_loss: 0.0173
Epoch 3/30
765/765 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9953 - loss: 0.0150 - val_accuracy: 0.9953 - val_loss: 0.0157
Epoch 4/30
765/765 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9979 - loss: 0.0083 - val_accuracy: 0.9956 - val_loss: 0.0151
Epoch 5/30
765/765 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9979 - loss: 0.0070 - val_accuracy: 0.9958 - val_loss: 0.0180
Epoch 6/30
765/765 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9989 - loss: 0.0040 - val_accuracy: 0.9879 - val_loss: 0.0511
Epoch 7/30
765/765 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9986 - loss: 0.0057 - val_accuracy: 0.9954 - val_loss: 0.0207
Epoch 8/30
765/765 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9992 - loss: 0.0024 - val_accuracy: 0.

In [8]:
#Print training/validation results for Model 1 at optimal epochs determined by ModelCheckpoint
test_model_1 = keras.models.load_model("Model_1.keras")
train_loss, train_acc = test_model_1.evaluate(train_images_1,train_labels_1)
test_loss, test_acc = test_model_1.evaluate(test_images_1,test_labels_1)
print("Model 1:")
print(f"Train accuracy: {train_acc:.3f}")
print(f"Test accuracy: {test_acc:.3f}")

765/765 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9990 - loss: 0.0043
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9974 - loss: 0.0081
Model 1:
Train accuracy: 0.999
Test accuracy: 0.998


In [9]:
#Model 2
inputs = keras.Input(shape=(28, 28, 1)) ## Different from densenet input
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model_2 = keras.Model(inputs=inputs, outputs=outputs)

model_2.compile(loss="SparseCategoricalCrossentropy", optimizer="rmsprop",
metrics=["accuracy"])

#Creating ModelCheckpoint callback
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="Model_2.keras",
save_best_only=True,
monitor="val_loss")
]

history = model_2.fit(train_images_2, train_labels_2,
epochs=30,
validation_data=(val_images_2,val_labels_2),
callbacks=callbacks)

Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.0542 - loss: 2.3161 - val_accuracy: 0.4000 - val_loss: 1.7672
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.3479 - loss: 1.8206 - val_accuracy: 0.4000 - val_loss: 1.5424
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.2938 - loss: 1.5722 - val_accuracy: 0.4000 - val_loss: 1.5355
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.5813 - loss: 1.3517 - val_accuracy: 0.4000 - val_loss: 1.3903
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5333 - loss: 1.2274 - val_accuracy: 0.6000 - val_loss: 1.1772
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.7500 - loss: 0.9129 - val_accuracy: 0.6000 - val_loss: 1.1626
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.6313 - loss: 1.0045 - val_accuracy: 0.7000 - val_loss: 0.9191
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9187 - loss: 0.6889 - val_accuracy: 0.7000 - val_loss: 1.0877
Ep

In [10]:
#Print training/validation results for Model 2 at optimal epochs determined by ModelCheckpoint
test_model_2 = keras.models.load_model("Model_2.keras")
train_loss, train_acc = test_model_2.evaluate(train_images_2,train_labels_2)
test_loss, test_acc = test_model_2.evaluate(test_images_2,test_labels_2)
print("Model 2:")
print(f"Train accuracy: {train_acc:.3f}")
print(f"Test accuracy: {test_acc:.3f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 826ms/step - accuracy: 1.0000 - loss: 0.0177
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6789 - loss: 1.2701
Model 2:
Train accuracy: 1.000
Test accuracy: 0.731


In [11]:
#Start of 8-2
#Steps: Load model 1 (covnet only), Retrain Dense Layer with training set 2,
#report training/test accuracy (with set 2)

#Load Model and Freeze Conv Layers:
model_1 = keras.models.load_model("Model_1.keras")

for layer in model_1.layers:
    if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.MaxPooling2D):
        layer.trainable = False

model_1.compile(loss="SparseCategoricalCrossentropy", optimizer="rmsprop",
metrics=["accuracy"])

#Creating ModelCheckpoint callback
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="Model_1(NoFineTuning).keras",
save_best_only=True,
monitor="val_loss")
]

In [12]:
#Train Model 1 on Set 2
history = model_1.fit(
    train_images_2, train_labels_2,
    epochs=30,
    validation_data=(val_images_2, val_labels_2),
    callbacks=callbacks
)

print("Model 1 On Set 2 (No fine-tuning):")
model_1 = keras.models.load_model("Model_1(NoFineTuning).keras")
#Training accuracy
train_loss, train_acc = model_1.evaluate(train_images_2, train_labels_2, verbose=0)
print(f"Training accuracy: {train_acc:.3f}")

#Test accuracy
test_loss, test_acc = model_1.evaluate(test_images_2, test_labels_2, verbose=0)
print(f"Test accuracy: {test_acc:.3f}")

Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.0000e+00 - loss: 10.8379 - val_accuracy: 0.0000e+00 - val_loss: 8.7056
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.0000e+00 - loss: 7.4325 - val_accuracy: 0.0000e+00 - val_loss: 6.5721
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.0542 - loss: 5.0777 - val_accuracy: 0.0000e+00 - val_loss: 4.7186
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.0708 - loss: 3.6043 - val_accuracy: 0.0000e+00 - val_loss: 3.5615
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.1146 - loss: 2.5959 - val_accuracy: 0.0000e+00 - val_loss: 2.8178
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.2771 - loss: 1.9125 - val_accuracy: 0.4000 - val_loss: 2.3215
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.4729 - loss: 1.4292 - val_accuracy: 0.4000 - val_loss: 1.9873
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.5500 - loss: 1.1300 - val_accuracy:

In [13]:
#Pretrain with fine-tuning
#Steps: Same as above + unfreeze the last k conv layers,
#retrain the whole model with training set 2. Try different k’s (1 - 3) and learning rates (you decide)
#k=1,lr=1e-3

model_1 = keras.models.load_model("Model_1.keras")

k = 1  #layers to unfreeze (Testing 1,2,3)
conv_layers = []

for layer in model_1.layers:
    if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.MaxPooling2D):
        conv_layers.append(layer)


conv_layers = [layer for layer in model_1.layers if isinstance(layer, keras.layers.Conv2D)]
#Unfreeze last k layers
for layer in conv_layers[-k:]:
    layer.trainable = True

#Testing learning rates 1e-3 and 1e-4
model_1.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.RMSprop(learning_rate=1e-3), metrics=["accuracy"])

callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="Model_1(k=1,lr=1e-3).keras",
save_best_only=True,
monitor="val_loss")
]

In [15]:
history = model_1.fit(
    train_images_2, train_labels_2,
    epochs=30,
    validation_data=(val_images_2, val_labels_2),
    callbacks=callbacks
)

print("Model 1 On Set 2, Fine-tuning, k=1,LR=1e-3):")
model_1 = keras.models.load_model("Model_1(k=1,lr=1e-3).keras")

#Training accuracy
train_loss, train_acc = model_1.evaluate(train_images_2, train_labels_2, verbose=0)
print(f"Training accuracy: {train_acc:.3f}")

#Test accuracy
test_loss, test_acc = model_1.evaluate(test_images_2, test_labels_2, verbose=0)
print(f"Test accuracy: {test_acc:.3f}")

Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.0000e+00 - loss: 10.2240 - val_accuracy: 0.0000e+00 - val_loss: 3.8732
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.0167 - loss: 3.2313 - val_accuracy: 0.3000 - val_loss: 2.1672
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.4292 - loss: 1.5886 - val_accuracy: 0.4000 - val_loss: 1.4826
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.8313 - loss: 0.8919 - val_accuracy: 0.5000 - val_loss: 1.1334
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.9187 - loss: 0.5742 - val_accuracy: 0.6000 - val_loss: 0.8903
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.9563 - loss: 0.3844 - val_accuracy: 0.7000 - val_loss: 0.8261
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 1.0000 - loss: 0.2648 - val_accuracy: 0.7000 - val_loss: 0.7400
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 1.0000 - loss: 0.1854 - val_accuracy: 0.7000 - val_loss: 

In [16]:
#k=2,lr=1e-3
model_1 = keras.models.load_model("Model_1.keras")

k = 2  #layers to unfreeze (Testing 1,2,3)
conv_layers = []

for layer in model_1.layers:
    if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.MaxPooling2D):
        conv_layers.append(layer)


conv_layers = [layer for layer in model_1.layers if isinstance(layer, keras.layers.Conv2D)]
#Unfreeze last k layers
for layer in conv_layers[-k:]:
    layer.trainable = True

#Testing learning rates 1e-3 and 1e-4
model_1.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.RMSprop(learning_rate=1e-3), metrics=["accuracy"])
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="Model_1(k=2,lr=1e-3).keras",
save_best_only=True,
monitor="val_loss")
]
history = model_1.fit(
    train_images_2, train_labels_2,
    epochs=30,
    validation_data=(val_images_2, val_labels_2),
    callbacks=callbacks
)
model_1 = keras.models.load_model("Model_1(k=2,lr=1e-3).keras")
print("Model 1 On Set 2, Fine-tuning, k=2,LR=1e-3):")
#Training accuracy
train_loss, train_acc = model_1.evaluate(train_images_2, train_labels_2, verbose=0)
print(f"Training accuracy: {train_acc:.3f}")

#Test accuracy
test_loss, test_acc = model_1.evaluate(test_images_2, test_labels_2, verbose=0)
print(f"Test accuracy: {test_acc:.3f}")

Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.0000e+00 - loss: 10.1751 - val_accuracy: 0.0000e+00 - val_loss: 3.9379
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 0.0542 - loss: 3.0641 - val_accuracy: 0.1000 - val_loss: 2.2630
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.5396 - loss: 1.5583 - val_accuracy: 0.3000 - val_loss: 1.6248
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.6958 - loss: 0.9464 - val_accuracy: 0.3000 - val_loss: 1.2827
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.8646 - loss: 0.6119 - val_accuracy: 0.6000 - val_loss: 0.9493
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.9729 - loss: 0.3928 - val_accuracy: 0.7000 - val_loss: 0.8242
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.9833 - loss: 0.2684 - val_accuracy: 0.7000 - val_loss: 0.7404
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 1.0000 - loss: 0.2106 - val_accuracy: 0.7000 - val_loss:

In [17]:
#k=3,lr=1e-3
model_1 = keras.models.load_model("Model_1.keras")

k = 3  #layers to unfreeze (Testing 1,2,3)
conv_layers = []

for layer in model_1.layers:
    if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.MaxPooling2D):
        conv_layers.append(layer)


conv_layers = [layer for layer in model_1.layers if isinstance(layer, keras.layers.Conv2D)]
#Unfreeze last k layers
for layer in conv_layers[-k:]:
    layer.trainable = True

#Testing learning rates 1e-3 and 1e-4
model_1.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.RMSprop(learning_rate=1e-3), metrics=["accuracy"])
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="Model_1(k=3,lr=1e-3).keras",
save_best_only=True,
monitor="val_loss")
]
history = model_1.fit(
    train_images_2, train_labels_2,
    epochs=30,
    validation_data=(val_images_2, val_labels_2),
    callbacks=callbacks
)
model_1 = keras.models.load_model("Model_1(k=3,lr=1e-3).keras")

print("Model 1 On Set 2, Fine-tuning, k=3,LR=1e-3):")
#Training accuracy
train_loss, train_acc = model_1.evaluate(train_images_2, train_labels_2, verbose=0)
print(f"Training accuracy: {train_acc:.3f}")

#Test accuracy
test_loss, test_acc = model_1.evaluate(test_images_2, test_labels_2, verbose=0)
print(f"Test accuracy: {test_acc:.3f}")

Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.0000e+00 - loss: 10.4483 - val_accuracy: 0.0000e+00 - val_loss: 4.1229
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.0500 - loss: 3.1660 - val_accuracy: 0.2000 - val_loss: 2.1564
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.4896 - loss: 1.3999 - val_accuracy: 0.4000 - val_loss: 1.4206
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.7937 - loss: 0.8437 - val_accuracy: 0.4000 - val_loss: 1.1283
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 1.0000 - loss: 0.5470 - val_accuracy: 0.7000 - val_loss: 0.9248
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 1.0000 - loss: 0.3511 - val_accuracy: 0.6000 - val_loss: 0.7996
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.9833 - loss: 0.2482 - val_accuracy: 0.7000 - val_loss: 0.6825
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 1.0000 - loss: 0.1725 - val_accuracy: 0.7000 - val_loss: 

In [18]:
#=1,lr=1e-4
model_1 = keras.models.load_model("Model_1.keras")

k = 1  #layers to unfreeze (Testing 1,2,3)
conv_layers = []

for layer in model_1.layers:
    if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.MaxPooling2D):
        conv_layers.append(layer)


conv_layers = [layer for layer in model_1.layers if isinstance(layer, keras.layers.Conv2D)]
#Unfreeze last k layers
for layer in conv_layers[-k:]:
    layer.trainable = True

#Testing learning rates 1e-3 and 1e-4
model_1.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.RMSprop(learning_rate=1e-4), metrics=["accuracy"])
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="Model_1(k=1,lr=1e-4).keras",
save_best_only=True,
monitor="val_loss")
]
history = model_1.fit(
    train_images_2, train_labels_2,
    epochs=30,
    validation_data=(val_images_2, val_labels_2),
    callbacks=callbacks
)
model_1 = keras.models.load_model("Model_1(k=1,lr=1e-4).keras")

print("Model 1 On Set 2, Fine-tuning, k=1,LR=1e-4):")
#Training accuracy
train_loss, train_acc = model_1.evaluate(train_images_2, train_labels_2, verbose=0)
print(f"Training accuracy: {train_acc:.3f}")

#Test accuracy
test_loss, test_acc = model_1.evaluate(test_images_2, test_labels_2, verbose=0)
print(f"Test accuracy: {test_acc:.3f}")

Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.0000e+00 - loss: 11.0153 - val_accuracy: 0.0000e+00 - val_loss: 11.3609
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.0000e+00 - loss: 9.8875 - val_accuracy: 0.0000e+00 - val_loss: 10.5160
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.0000e+00 - loss: 9.2933 - val_accuracy: 0.0000e+00 - val_loss: 9.9874
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.0000e+00 - loss: 8.7484 - val_accuracy: 0.0000e+00 - val_loss: 9.5286
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.0000e+00 - loss: 8.2068 - val_accuracy: 0.0000e+00 - val_loss: 9.0849
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.0000e+00 - loss: 7.6023 - val_accuracy: 0.0000e+00 - val_loss: 8.6349
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.0000e+00 - loss: 7.2852 - val_accuracy: 0.0000e+00 - val_loss: 8.2877
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.0000e

In [19]:
#k=2,lr=1e-4
model_1 = keras.models.load_model("Model_1.keras")

k = 2  #layers to unfreeze (Testing 1,2,3)
conv_layers = []

for layer in model_1.layers:
    if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.MaxPooling2D):
        conv_layers.append(layer)


conv_layers = [layer for layer in model_1.layers if isinstance(layer, keras.layers.Conv2D)]
#Unfreeze last k layers
for layer in conv_layers[-k:]:
    layer.trainable = True

#Testing learning rates 1e-3 and 1e-4
model_1.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.RMSprop(learning_rate=1e-4), metrics=["accuracy"])
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="Model_1(k=2,lr=1e-4).keras",
save_best_only=True,
monitor="val_loss")
]
history = model_1.fit(
    train_images_2, train_labels_2,
    epochs=30,
    validation_data=(val_images_2, val_labels_2),
    callbacks=callbacks
)
model_1 = keras.models.load_model("Model_1(k=2,lr=1e-4).keras")

print("Model 1 On Set 2, Fine-tuning, k=2,LR=1e-4):")
#Training accuracy
train_loss, train_acc = model_1.evaluate(train_images_2, train_labels_2, verbose=0)
print(f"Training accuracy: {train_acc:.3f}")

#Test accuracy
test_loss, test_acc = model_1.evaluate(test_images_2, test_labels_2, verbose=0)
print(f"Test accuracy: {test_acc:.3f}")

Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.0000e+00 - loss: 10.9257 - val_accuracy: 0.0000e+00 - val_loss: 11.2513
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.0000e+00 - loss: 9.8132 - val_accuracy: 0.0000e+00 - val_loss: 10.4945
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.0000e+00 - loss: 9.0340 - val_accuracy: 0.0000e+00 - val_loss: 9.8627
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.0000e+00 - loss: 8.5047 - val_accuracy: 0.0000e+00 - val_loss: 9.3864
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.0000e+00 - loss: 8.0405 - val_accuracy: 0.0000e+00 - val_loss: 8.9411
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.0000e+00 - loss: 7.6797 - val_accuracy: 0.0000e+00 - val_loss: 8.5563
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.0000e+00 - loss: 7.3635 - val_accuracy: 0.0000e+00 - val_loss: 8.2298
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.0000e

In [20]:
#k=3,lr=1e-4
model_1 = keras.models.load_model("Model_1.keras")

k = 3  #layers to unfreeze (Testing 1,2,3)
conv_layers = []

for layer in model_1.layers:
    if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.MaxPooling2D):
        conv_layers.append(layer)


conv_layers = [layer for layer in model_1.layers if isinstance(layer, keras.layers.Conv2D)]
#Unfreeze last k layers
for layer in conv_layers[-k:]:
    layer.trainable = True

#Testing learning rates 1e-3 and 1e-4
model_1.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.RMSprop(learning_rate=1e-4), metrics=["accuracy"])
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="Model_1(k=3,lr=1e-4).keras",
save_best_only=True,
monitor="val_loss")
]
history = model_1.fit(
    train_images_2, train_labels_2,
    epochs=30,
    validation_data=(val_images_2, val_labels_2),
    callbacks=callbacks
)
model_1 = keras.models.load_model("Model_1(k=3,lr=1e-4).keras")

print("Model 1 On Set 2, Fine-tuning, k=3,LR=1e-4):")
#Training accuracy
train_loss, train_acc = model_1.evaluate(train_images_2, train_labels_2, verbose=0)
print(f"Training accuracy: {train_acc:.3f}")

#Test accuracy
test_loss, test_acc = model_1.evaluate(test_images_2, test_labels_2, verbose=0)
print(f"Test accuracy: {test_acc:.3f}")

Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.0000e+00 - loss: 11.0401 - val_accuracy: 0.0000e+00 - val_loss: 11.2914
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.0000e+00 - loss: 9.8932 - val_accuracy: 0.0000e+00 - val_loss: 10.5455
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.0000e+00 - loss: 9.1843 - val_accuracy: 0.0000e+00 - val_loss: 9.9801
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.0000e+00 - loss: 8.6572 - val_accuracy: 0.0000e+00 - val_loss: 9.4557
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.0000e+00 - loss: 8.2089 - val_accuracy: 0.0000e+00 - val_loss: 9.0249
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.0000e+00 - loss: 7.5844 - val_accuracy: 0.0000e+00 - val_loss: 8.6149
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.0000e+00 - loss: 7.2063 - val_accuracy: 0.0000e+00 - val_loss: 8.2029
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.0000e

In [ ]:
'''The Final Results are (Yours may be slightly different):
No Fine Tuning -
Training accuracy: 1.000
Test accuracy: 0.820

Model 1 On Set 2, Fine-tuning, k=1,LR=1e-3):
Training accuracy: 1.000
Test accuracy: 0.793

Model 1 On Set 2, Fine-tuning, k=2,LR=1e-3):
Training accuracy: 1.000
Test accuracy: 0.786

Model 1 On Set 2, Fine-tuning, k=3,LR=1e-3):
Training accuracy: 1.000
Test accuracy: 0.795

Model 1 On Set 2, Fine-tuning, k=1,LR=1e-4):
Training accuracy: 0.150
Test accuracy: 0.059

Model 1 On Set 2, Fine-tuning, k=2,LR=1e-4):
Training accuracy: 0.150
Test accuracy: 0.063

Model 1 On Set 2, Fine-tuning, k=3,LR=1e-4):
Training accuracy: 0.200
Test accuracy: 0.078 '''
#As we can see, The no-fine tuning model did the best.
#Followed closely behind is the fine-tuned models with lr=1e-3.
#Far behind in last place is the fine-tuned models with lr=1e-4.

#Intuitively, I thought the fine-tuned models would do better, and I was surprised
#to see that the original model did the best.
#The weights developed by the first model on the first training set must have been very
#similiar to the weights developed on a model training solely on the 2nd set. This
#could be due to the line like nature of the digits, or how the white lines contrast
#against the black or a combination of the two.